In [30]:
import spacy

In [31]:
nlp=spacy.load("en_core_web_sm")

In [32]:
doc=nlp("Donald Trump is the President of the USA")
doc.ents

(Donald Trump, USA)

In [33]:
doc.ents[0],type(doc.ents[0])

(Donald Trump, spacy.tokens.span.Span)

In [34]:
from spacy import displacy
displacy.render(doc,style='ent',jupyter=True)

In [35]:
spacy.explain('GPE')

'Countries, cities, states'

In [36]:
import json
with open("./Corona2.json",'r') as f:
    data=json.load(f)

In [41]:
data['examples'][0]

{'id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
 'content': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]",
 'metadata': {},
 'annotations': [{'id': '0825a1

In [43]:
data['examples'][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [44]:
data['examples'][0]['annotations'][0]

{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
 'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
 'end': 371,
 'start': 360,
 'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
 'tag_name': 'Medicine',
 'value': 'Diosmectite',
 'correct': None,
 'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
   'annotator_id': 1,
   'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
   'name': 'Ashpat123',
   'reason': 'exploration'}],
 'model_annotations': []}

In [46]:
data['examples'][0]['content']

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [47]:
training_data=[]
for example in data['examples']:
    temp_dict={}
    temp_dict['text']=example['content']
    temp_dict['entities']=[]
    for annotation in example['annotations']:
        start=annotation['start']
        end=annotation['end']
        label=annotation['tag_name'].upper()
        temp_dict['entities'].append((start,end,label))
    training_data.append(temp_dict)

In [52]:
training_data[0]

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]",
 'entities': [(360, 371, 'MEDICINE'),
  (383, 408, 'MEDICINE'),
  (104, 112, 'MEDICALCONDITION'),


In [57]:
training_data[0]['text'][383:408]

'aluminomagnesium silicate'

In [58]:
from spacy.tokens import DocBin
from tqdm import tqdm

In [59]:
nlp=spacy.blank('en') #loading new spacy model
doc_bin=DocBin()

In [60]:
from spacy.util import filter_spans

In [ ]:
for training_example in tqdm(training_data):
    text=training_example['text']
    labels=training_example['entities']
    doc=nlp.make_doc(text)
    ents=[]
    for start,end,label in labels:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents=filter_spans(ents)
    doc.ents=filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk('train.spacy')

Now we need a config file

In [62]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [64]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    150.79    0.00    0.00    0.00    0.00
  7     200       1148.68   3300.10   81.38   81.05   81.71    0.81
 14     400        225.00    665.43   96.13   96.33   95.93    0.96
 22     600       1143.99    287.99   96.91   98.33   95.53    0.97
 30     800        219.41    236.21   97.17   96.77   97.56    0.97
 40    1000       4487.30    228.55   98.37   98.37   98.37    0.98
 52    1200        221.40    203.49   98.17   97.98   98.37    0.98
 65    1400        312.85    217.40   97.96   98.36   97.56    0.98
 82    1600        333.04    229.42 

[2022-12-28 17:52:38,839] [INFO] Set up nlp object from config
[2022-12-28 17:52:38,860] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-12-28 17:52:38,864] [INFO] Created vocabulary
[2022-12-28 17:52:38,866] [INFO] Finished initializing nlp object
[2022-12-28 17:52:39,285] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [65]:
nlp_ner=spacy.load("model-best")

In [72]:
training_data[1]['text']

'Diarrhea, also spelled diarrhoea, is the condition of having at least three loose, liquid, or watery bowel movements each day.[2] It often lasts for a few days and can result in dehydration due to fluid loss.[2] Signs of dehydration often begin with loss of the normal stretchiness of the skin and irritable behaviour.[2] This can progress to decreased urination, loss of skin color, a fast heart rate, and a decrease in responsiveness as it becomes more severe.[2] Loose but non-watery stools in babies who are exclusively breastfed, however, are normal.[2]'

In [73]:
doc=nlp_ner(training_data[1]['text'])

In [76]:
colors={"PATHOGEN":"#DC0000",'MEDICINE':"#205295","MEDICALCONDITION":"#FF8E9E"}
options={"colors":colors}
displacy.render(doc,style='ent',options=options,jupyter=True)

In [77]:
doc=nlp_ner(training_data[2]['text'])
displacy.render(doc,style='ent',options=options,jupyter=True)